In [179]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import random
import warnings
warnings.filterwarnings("ignore")

In [335]:
columns = []
data = []
test_req = requests.get("https://www.pro-football-reference.com/boxscores/201710220chi.htm")
soup = BeautifulSoup(test_req.content, "html.parser")
scoring_table = soup.find_all('h2')[0]
if 'Scoring' not in str(scoring_table):
    print('Error')
test_table = scoring_table.find_next('table')
if columns == []:
    columns = [th.text.strip() for th in test_table.find_all('th')][:6]
elif columns[2:] != [th.text.strip() for th in test_table.find_all('th')]:
    print(f'issue in season {season} week {week}' )
for row in test_table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

In [337]:
pd.DataFrame(data, columns=columns)

,Quarter,Time,Tm,Detail,CAR,CHI
0,1,8:59,Bears,Eddie Jackson 75 yard fumble return (Connor Ba...,0,7
1,2,12:28,Bears,Eddie Jackson 76 yard interception return (Con...,0,14
2,,3:18,Panthers,Graham Gano 36 yard field goal,3,14
3,,1:10,Bears,Connor Barth 19 yard field goal,3,17


In [338]:
def clean_df(df):
    df = df.drop(columns=['Tm','Detail'])

    df['num_seconds'] = df['Time'].str.split(":").str[0].astype(int)*60+df['Time'].str.split(":").str[1].astype(int)
    
    index_list = []
    if '1' in df['Quarter'].unique():
        index_1 = df.loc[df['Quarter']=='1',:].index[0]
        index_list.append(index_1)
    if '2' in df['Quarter'].unique():
        index_2 = df.loc[df['Quarter']=='2',:].index[0]
        index_list.append(index_2)
    if '3' in df['Quarter'].unique():
        index_3 = df.loc[df['Quarter']=='3',:].index[0]
        index_list.append(index_3)
    if '4' in df['Quarter'].unique():
        index_4 = df.loc[df['Quarter']=='4',:].index[0]
        index_list.append(index_4)
        
    if '1' in df['Quarter'].unique():
        if index_1 == max(index_list):
            lowest_num = len(df)
        else:
            lowest_num = min([x for x in index_list if x > index_1])
        df.loc[index_1:lowest_num-1,'num_seconds'] = df.loc[index_1:lowest_num-1,'num_seconds'] + 15*60*3
    if '2' in df['Quarter'].unique():
        if index_2 == max(index_list):
            lowest_num = len(df)
        else:
            lowest_num = min([x for x in index_list if x > index_2])
        df.loc[index_2:lowest_num-1,'num_seconds'] = df.loc[index_2:lowest_num-1,'num_seconds'] + 15*60*2
    if '3' in df['Quarter'].unique():
        if index_3 == max(index_list):
            lowest_num = len(df)
        else:
            lowest_num = min([x for x in index_list if x > index_3])
        df.loc[index_3:lowest_num-1,'num_seconds'] = df.loc[index_3:lowest_num-1,'num_seconds'] + 15*60*1

    df.iloc[:,2] = df.iloc[:,2].astype(int)
    df.iloc[:,3] = df.iloc[:,3].astype(int)

    df['within_3'] = (abs(df.iloc[:,3] - df.iloc[:,2])<=3).astype(int)
    return df

df = clean_df(pd.DataFrame(data, columns=columns))
df

,Quarter,Time,CAR,CHI,num_seconds,within_3
0,1,8:59,0,7,3239,0
1,2,12:28,0,14,2548,0
2,,3:18,3,14,1998,0
3,,1:10,3,17,1870,0


In [275]:
def calculate_prop(df):
    secs_remaining = 3600
    num = 0
    counting = True
    for i in range(len(df)):
        if counting:
            if df.iloc[i,:]['within_3'] == 1:
                continue
            else:
                num = num + (secs_remaining - df.iloc[i,:]['num_seconds'])
                counting = False
                secs_remaining = df.iloc[i,:]['num_seconds']
        else:
            if df.iloc[i,:]['within_3'] == 0:
                continue
            else:
                counting = True
                secs_remaining = df.iloc[i,:]['num_seconds']
    if counting:
        num = num + secs_remaining
    prop = num/3600
    return prop
calculate_prop(df)

0.4438888888888889

In [276]:
def calculate_switches(df):
    df['differential'] = df.iloc[:,2] - df.iloc[:,3]
    
    last_sign = 1
    sign_changes = 0

    for x in df['differential']:
        if x == 0:
            sign = 0
        else:
            sign = x / abs(x)

        if sign != last_sign:
            sign_changes = sign_changes + 1
            last_sign = sign
    return sign_changes
calculate_switches(df)

3

In [379]:
full_df = pd.read_csv("../data/closeness_links.csv")
full_df

,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,Underdog,Over/Under,Notes,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,Subscribers (in thousands)
0,2017,1,Thu,7-Sep-17,8:30,@,New England Patriots,L 27-42,L -8,NaN,Kansas City Chiefs,O 47.5,NaN,NaN,NaN,NaN,https://www.reddit.com/r/nfl/comments/6yr619/g...,https://www.reddit.com/r/nfl/comments/6ysc10/p...,640
1,2017,1,Sun,10-Sep-17,1:00,@,Buffalo Bills,W 21-12,W -7,NaN,New York Jets,U 42,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97il/g...,https://www.reddit.com/r/nfl/comments/6zajs1/p...,640
2,2017,1,Sun,10-Sep-17,1:00,NaN,Atlanta Falcons,W 23-17,L -6.5,@,Chicago Bears,U 48,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97gn/g...,https://www.reddit.com/r/nfl/comments/6zajuc/p...,640
3,2017,1,Sun,10-Sep-17,1:00,@,Cincinnati Bengals,L 0-20,L -2.5,NaN,Baltimore Ravens,U 41.5,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97hl/g...,https://www.reddit.com/r/nfl/comments/6zajr0/p...,640
4,2017,1,Sun,10-Sep-17,1:00,NaN,Pittsburgh Steelers,W 21-18,L -10,@,Cleveland Browns,U 47,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97l9/g...,https://www.reddit.com/r/nfl/comments/6zajwi/p...,640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2022,18,Sun,8-Jan-23,4:25,@,Philadelphia Eagles,W 22-16,L -16,NaN,New York Giants,U 42,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u055/...,https://www.reddit.com/r/nfl/comments/106zb92/...,3197
1563,2022,18,Sun,8-Jan-23,4:25,@,San Francisco 49ers,W 38-13,W -14,NaN,Arizona Cardinals,O 39.5,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u07e/...,https://www.reddit.com/r/nfl/comments/106z77a/...,3197
1564,2022,18,Sun,8-Jan-23,4:25,@,Seattle Seahawks,W 19-16 (OT),L -6,NaN,Los Angeles Rams,U 41.5,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u06w/...,https://www.reddit.com/r/nfl/comments/106zix9/...,3197
1565,2022,18,Sun,8-Jan-23,4:25,NaN,Dallas Cowboys,L 6-26,L -7,@,Washington Commanders,U 40,NaN,https://www.reddit.com/r/nfl/comments/106o2qc/...,1.0,1.0,https://www.reddit.com/r/nfl/comments/106u05s/...,https://www.reddit.com/r/nfl/comments/106zgna/...,3197


In [380]:
full_df['Date'] = pd.to_datetime(full_df['Date'])
nfl_teams_code = {
    'arizona cardinals': 'crd',
    'atlanta falcons': 'atl',
    'baltimore ravens': 'rav',
    'buffalo bills': 'buf',
    'carolina panthers': 'car',
    'chicago bears': 'chi',
    'cincinnati bengals': 'cin',
    'cleveland browns': 'cle',
    'dallas cowboys': 'dal',
    'denver broncos': 'den',
    'detroit lions': 'det',
    'green bay packers': 'gnb',
    'houston texans': 'htx',
    'indianapolis colts': 'clt',
    'jacksonville jaguars': 'jax',
    'kansas city chiefs': 'kan',
    'las vegas raiders': 'rai',
    'los angeles chargers': 'sdg',
    'los angeles rams': 'ram',
    'miami dolphins': 'mia',
    'minnesota vikings': 'min',
    'new england patriots': 'nwe',
    'new orleans saints': 'nor',
    'new york giants': 'nyg',
    'new york jets': 'nyj',
    'oakland raiders': 'rai',
    'philadelphia eagles': 'phi',
    'pittsburgh steelers': 'pit',
    'san francisco 49ers': 'sfo',
    'seattle seahawks': 'sea',
    'tampa bay buccaneers': 'tam',
    'tennessee titans': 'oti',
    'washington football team': 'was',
    'washington commanders': 'was',
    'washington redskins': 'was'
}
full_df['link_text_fav'] = "https://www.pro-football-reference.com/boxscores/" + \
                    full_df['Date'].dt.strftime('%Y%m%d').astype(str) + '0' + \
                    full_df['Favorite'].str.lower().replace(nfl_teams_code) + ".htm"
full_df['link_text_dog'] = "https://www.pro-football-reference.com/boxscores/" + \
                    full_df['Date'].dt.strftime('%Y%m%d').astype(str) + '0' + \
                    full_df['Underdog'].str.lower().replace(nfl_teams_code) + ".htm"

In [315]:
# problematic_rows = []
# props = []
# switches = []
for index, row in full_df.iterrows():
    if index % 100 == 0:
        print(f"Scraped {index}")
    test_req = requests.get(row['link_text_fav'])
    soup = BeautifulSoup(test_req.content, "html.parser")
    scoring_table = soup.find_all('h2')[0]
    if 'Scoring' not in str(scoring_table):
        test_req = requests.get(row['link_text_dog'])
        soup = BeautifulSoup(test_req.content, "html.parser")
        scoring_table = soup.find_all('h2')[0]
        if 'Scoring' not in str(scoring_table):
            problematic_rows.append(index)
            props.append(np.NaN)
            switches.append(np.NaN)
            time.sleep(3)
            continue
    try:
        test_table = scoring_table.find_next('table')
        columns = [th.text.strip() for th in test_table.find_all('th')][:6]
        data = []
        for row in test_table.find('tbody').find_all('tr'):
            row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
            data.append(row_data)
        temp_df = pd.DataFrame(data, columns=columns)
        
        temp_df = clean_df(temp_df)
        
        props.append(calculate_prop(temp_df))
        
        switches.append(calculate_switches(temp_df))
        
    except:
        problematic_rows.append(index)
        props.append(np.NaN)        
    time.sleep(3)

Scraped 0
Scraped 100
Scraped 200
Scraped 300
Scraped 400
Scraped 500
Scraped 600
Scraped 700
Scraped 800
Scraped 900
Scraped 1000


KeyboardInterrupt: 

In [342]:
[x for x in problematic_rows if x < 700]
# tenessee titans -> oti?

[7,
 41,
 90,
 93,
 126,
 139,
 180,
 185,
 188,
 233,
 255,
 280,
 312,
 336,
 337,
 346,
 398,
 406,
 427,
 437,
 443,
 448,
 480,
 511,
 537,
 539,
 567,
 584,
 613,
 623,
 627,
 654,
 684]

In [378]:
props[7]

nan

In [365]:
full_df.loc[([x for x in problematic_rows if x < 700]),:].loc[(full_df['Favorite']!='Tennessee Titans'),:]\
                                    .loc[(full_df['Underdog']!='Tennessee Titans'),:]


,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,...,Over/Under,Notes,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,Subscribers (in thousands),link_text_fav,link_text_dog
93,2017,7,Sun,2017-10-22,1:00,NaN,Carolina Panthers,L 3-17,L -2.5,@,...,U 39.5,NaN,https://www.reddit.com/r/nfl/comments/7810f7/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/7817bq/g...,https://www.reddit.com/r/nfl/comments/782hos/p...,654,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
180,2017,13,Sun,2017-12-03,1:00,@,Chicago Bears,L 14-15,L -2.5,NaN,...,U 42,NaN,https://www.reddit.com/r/nfl/comments/7hb4ly/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/7haxlk/g...,https://www.reddit.com/r/nfl/comments/7hcb1z/p...,700,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
188,2017,13,Sun,2017-12-03,4:25,@,New Orleans Saints,W 31-21,W -5.5,NaN,...,O 48,NaN,https://www.reddit.com/r/nfl/comments/7hb4ly/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/7hcau4/g...,https://www.reddit.com/r/nfl/comments/7hdn6c/p...,700,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
336,2018,6,Sun,2018-10-14,1:00,@,Cincinnati Bengals,L 21-28,L -1.5,NaN,...,U 49.5,NaN,https://www.reddit.com/r/nfl/comments/9o3t58/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/9o42dm/g...,https://www.reddit.com/r/nfl/comments/9o5wtx/p...,848,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
337,2018,6,Sun,2018-10-14,1:00,@,Cleveland Browns,L 14-38,L -1,NaN,...,O 47,NaN,https://www.reddit.com/r/nfl/comments/9o3t58/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/9o428z/g...,https://www.reddit.com/r/nfl/comments/9o5wnv/p...,848,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
406,2018,11,Sun,2018-11-18,1:00,@,Baltimore Ravens,W 24-21,L -6.5,NaN,...,O 44,NaN,https://www.reddit.com/r/nfl/comments/9y7k93/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/9y7tya/g...,https://www.reddit.com/r/nfl/comments/9y9tn9/p...,871,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
427,2018,12,Sun,2018-11-25,4:05,@,Los Angeles Chargers,W 45-10,W -14,NaN,...,O 43,NaN,https://www.reddit.com/r/nfl/comments/a0a37x/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/a0c3lg/g...,https://www.reddit.com/r/nfl/comments/a0ebbg/p...,871,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
437,2018,13,Sun,2018-12-02,1:00,@,Houston Texans,W 29-13,W -5,NaN,...,U 47.5,NaN,https://www.reddit.com/r/nfl/comments/a2evb3/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/a2f5no/g...,https://www.reddit.com/r/nfl/comments/a2hap7/p...,916,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
539,2019,2,Sun,2019-09-15,4:05,NaN,Kansas City Chiefs,W 28-10,W -7,@,...,U 53.5,NaN,https://www.reddit.com/r/nfl/comments/d4mcds/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/d4palq/g...,https://www.reddit.com/r/nfl/comments/d4sapl/p...,1464,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...
567,2019,4,Sun,2019-09-29,1:00,NaN,Los Angeles Chargers,W 30-10,W -15,@,...,U 43.5,NaN,https://www.reddit.com/r/nfl/comments/dax11w/g...,1.0,1.0,https://www.reddit.com/r/nfl/comments/daxfut/g...,https://www.reddit.com/r/nfl/comments/db09hq/p...,1464,https://www.pro-football-reference.com/boxscor...,https://www.pro-football-reference.com/boxscor...


In [375]:
full_df.iloc[684,:]['link_text_fav']

'https://www.pro-football-reference.com/boxscores/201911240ten.htm'

In [373]:
row = full_df.iloc[623,:]#['link_text_fav'], full_df.iloc[1415,:]['link_text_dog']
test_req = requests.get(row['link_text_fav'])
soup = BeautifulSoup(test_req.content, "html.parser")
scoring_table = soup.find_all('h2')[0]
if 'Scoring' not in str(soup.find_all('h2')[0]):
    test_req = requests.get(row['link_text_dog'])
    soup = BeautifulSoup(test_req.content, "html.parser")
    scoring_table = soup.find_all('h2')[0]
    if 'Scoring' not in str(scoring_table):
        print('x')
test_table = scoring_table.find_next('table')
columns = [th.text.strip() for th in test_table.find_all('th')][:6]
data = []
for row in test_table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)
temp_df = pd.DataFrame(data, columns=columns)
temp_df = clean_df(temp_df)
temp_df
calculate_prop(temp_df)
calculate_switches(temp_df)

1

In [ ]:
for num in [x for x in problematic_rows if x < 700]:
    test_req = requests.get(full_df.loc[num,'link_text_fav'])
    soup = BeautifulSoup(test_req.content, "html.parser")
    scoring_table = soup.find_all('h2')[0]
    if 'Scoring' not in str(scoring_table):
        test_req = requests.get(full_df.loc[num,'link_text_dog'])
        soup = BeautifulSoup(test_req.content, "html.parser")
        scoring_table = soup.find_all('h2')[0]
        if 'Scoring' not in str(scoring_table):
            problematic_rows.append(index)
            props.append(np.NaN)
            switches.append(np.NaN)
            time.sleep(3)
            continue
    try:
        test_table = scoring_table.find_next('table')
        columns = [th.text.strip() for th in test_table.find_all('th')][:6]
        data = []
        for row in test_table.find('tbody').find_all('tr'):
            row_data = [td.text.strip() for td in row.find_all('th')] + [td.text.strip() for td in row.find_all('td')]
            data.append(row_data)
        temp_df = pd.DataFrame(data, columns=columns)
        
        temp_df = clean_df(temp_df)
        
        props[num] = calculate_prop(temp_df)
        
        switches[num] = calculate_switches(temp_df)
        
        problematic_rows.remove(num)
        
    except:
        # problematic_rows.append(index)
        # props.append(np.NaN)   
        continue
    #time.sleep(random.uniform(2.5,3.5))